In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os import mkdir
from os.path import isdir
from PIL import Image
import cv2


In [ ]:
! kaggle 

usage: kaggle [-h] [-v] {competitions,c,datasets,d,kernels,k,config} ...
kaggle: error: too few arguments


In [ ]:
! cp kaggle.json /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d mrshukla/twice-members-faces

 84% 76.0M/89.9M [00:00<00:00, 67.1MB/s]
100% 89.9M/89.9M [00:00<00:00, 103MB/s] 


In [ ]:
! unzip -qq twice-members-faces.zip

In [ ]:
def extract_face(filename):
  image = cv2.imread(filename)
  return image.reshape(1,160,160,3)
 
# load images and extract faces for all images in a directory
def load_faces(directory):
  faces = np.zeros(shape=(0, 160, 160, 3))
  # enumerate files
  for filename in listdir(directory):
    # path
    path = directory + filename
    # get face
    face = extract_face(path)
    # store
#     print(faces.shape)
    faces = np.vstack((faces,face))
  return faces
 
# load a dataset that contains one subdir for each class that in turn contains images
def load_dataset(directory):
  X, y = np.zeros(shape=(0, 160, 160, 3)), list()
  # enumerate folders, on per class
  for subdir in listdir(directory):
    # path
    path = directory + subdir + '/'
    # skip any files that might be in the dir
    if not isdir(path):
      continue
    # load all faces in the subdirectory
    faces = load_faces(path)
    # create labels
    labels = [subdir for _ in range(len(faces))]
    # summarize progress
    print('>loaded %d examples for class: %s' % (len(faces), subdir))
    # store
    # 		X.extend(faces)
    X = np.vstack((X,faces))
    y.extend(labels)
  return X, np.asarray(y)
 
# load train dataset
aX, ay = load_dataset('faces/')
print(aX.shape, ay.shape)
# bX= np.asarray(aX)
# load test dataset
# testX, testy = load_dataset('5-celebrity-faces-dataset/val/')
# save arrays to one file in compressed format
# savez_compressed('5-celebrity-faces-dataset.npz', trainX, trainy, testX, testy)

>loaded 1840 examples for class: Minatozaki_Sana
>loaded 490 examples for class: Myoui_Mina
>loaded 74 examples for class: Park_Jihyo
>loaded 590 examples for class: Kim_Dahyun
>loaded 77 examples for class: Son_Chaeyoung
>loaded 1836 examples for class: Chou_Tzuyu
>loaded 1020 examples for class: Hirai_Momo
>loaded 620 examples for class: Im_Nayeon
>loaded 1416 examples for class: Yoo_Jeongyeon
(7963, 160, 160, 3) (7963,)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
...     aX, ay, test_size=0.20, random_state=42)

In [ ]:
np.savez_compressed('twice-members-faces.npz', X_train, y_train, X_test, y_test)

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model

In [ ]:
# from numpy import load
# from numpy import expand_dims
# from numpy import asarray
# from numpy import savez_compressed

In [ ]:
def get_embedding(model, face_pixels):
	# scale pixel values
	face_pixels = face_pixels.astype('float32')
	# standardize pixel values across channels (global)
	mean, std = face_pixels.mean(), face_pixels.std()
	face_pixels = (face_pixels - mean) / std
	# transform face into one sample
	samples = np.expand_dims(face_pixels, axis=0)
	# make prediction to get embedding
	yhat = model.predict(samples)
	return yhat[0]

In [ ]:
data = np.load('twice-members-faces.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
print('Loaded: ', trainX.shape, trainy.shape, testX.shape, testy.shape)
# load the facenet model
model = load_model('/content/drive/My Drive/keras-facenet/model/facenet_keras.h5')
print('Loaded Model')
# convert each face in the train set to an embedding
newTrainX = list()
for face_pixels in trainX:
	embedding = get_embedding(model, face_pixels)
	newTrainX.append(embedding)
newTrainX = np.asarray(newTrainX)
print(newTrainX.shape)
# convert each face in the test set to an embedding
newTestX = list()
for face_pixels in testX:
	embedding = get_embedding(model, face_pixels)
	newTestX.append(embedding)
newTestX = np.asarray(newTestX)
print(newTestX.shape)
# save arrays to one file in compressed format

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
np.savez_compressed('twice-members-faces-embeddings.npz', newTrainX, trainy, newTestX, testy)

NameError: ignored

In [ ]:
# from numpy import load
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

from random import choice
# from numpy import expand_dims
from google.colab.patches import cv2_imshow

In [ ]:
data = np.load('/content/drive/My Drive/twice_npy/twice-members-faces-embeddings.npz')
trainX, trainy, testX, testy = data['arr_0'], data['arr_1'], data['arr_2'], data['arr_3']
del data
print('Dataset: train=%d, test=%d' % (trainX.shape[0], testX.shape[0]))
# normalize input vectors
in_encoder = Normalizer(norm='l2')
trainX = in_encoder.transform(trainX)
testX = in_encoder.transform(testX)
# label encode targets
out_encoder = LabelEncoder()
out_encoder.fit(trainy)
trainy = out_encoder.transform(trainy)
testy = out_encoder.transform(testy)
# fit model
model = SVC(kernel='linear', probability=True)
model.fit(trainX, trainy)
# predict
# yhat_train = model.predict(trainX)
# yhat_test = model.predict(testX)
# # score
# score_train = accuracy_score(trainy, yhat_train)
# score_test = accuracy_score(testy, yhat_test)
# # summarize
# print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Dataset: train=6370, test=1593


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=True, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [ ]:
data = np.load('twice-members-faces.npz')
testX_faces = data['arr_2']
del data

In [ ]:
i = 0
s = [k for k in range(testX.shape[0])]
for selection in s :
  random_face_class = testy[selection]
  if random_face_class in {7} :
    random_face_pixels = testX_faces[selection]
    random_face_emb = testX[selection]
    print(testX.shape)
  #   print(out_encoder.classes_)
    random_face_name = out_encoder.inverse_transform([random_face_class])
#     print((random_face_class, random_face_name))
    # prediction for the face
    samples = np.expand_dims(random_face_emb, axis=0)
    print(samples.shape)
    break
    yhat_class = model.predict(samples)
    yhat_prob = model.predict_proba(samples)
    # get name
    class_index = yhat_class[0]
    class_probability = yhat_prob[0,class_index] * 100
    predict_names = out_encoder.inverse_transform(yhat_class)
    print('Predicted: %s (%.3f)' % (predict_names[0], class_probability))
    print('Expected: %s' % random_face_name[0])
    # plot for fun

    title = '%s (%.3f)' % (predict_names[0], class_probability)
    cv2_imshow(random_face_pixels)
    i=i+1
  if i==20 :
    break
  
#   print(predict_names)

(1593, 128)
(1, 128)


In [ ]:
out_encoder.classes_

array(['Chou_Tzuyu', 'Hirai_Momo', 'Im_Nayeon', 'Kim_Dahyun',
       'Minatozaki_Sana', 'Myoui_Mina', 'Park_Jihyo', 'Son_Chaeyoung',
       'Yoo_Jeongyeon'], dtype='<U15')

In [ ]:
from sklearn.externals import joblib

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [ ]:
filename = 'twice_members_2.sav'
joblib.dump(model, filename)

['twice_members_2.sav']

In [ ]:
loaded_model = joblib.load(filename)
result = loaded_model.score(testX, testy)

In [ ]:
result

0.871939736346516

In [ ]:
model.score(testX,testy)

0.871939736346516

In [ ]:
# # !pip install -U -q PyDrive
# # !git clone https://github.com/Joshua1989/python_scientific_computing.git
# !git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
# !mv /tmp/colab_util_repo/colab_util.py colab_util.py 
# !rm -r /tmp/colab_util_repo

Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.


In [ ]:
# from colab_util import *
# drive_handler = GoogleDriveHandler()

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# test_folder_id = drive_handler.create_folder('test_folder')
# test_folder_id

'1DVcnOiiYJ6bHeS8nIU32Gc9dSxOlaNeb'

In [ ]:
# drive_handler.upload("/content/twice-members-faces-embeddings.npz", parent_path='test_folder/')
# drive_handler.list_folder(test_folder_id, max_depth=1)

[{'id': '1RzCHNpJsu_hpEhzUHx5eynegOR46WjEC',
  'link': 'https://drive.google.com/file/d/1RzCHNpJsu_hpEhzUHx5eynegOR46WjEC/view?usp=drivesdk',
  'mimeType': 'application/x-zip',
  'title': 'twice-members-faces-embeddings.npz'},
 {'id': '1VuW08X2FQEOhhSg3ElJlfUAkL49nkIaH',
  'link': 'https://drive.google.com/file/d/1VuW08X2FQEOhhSg3ElJlfUAkL49nkIaH/view?usp=drivesdk',
  'mimeType': 'application/x-zip',
  'title': 'twice-members-faces.npz'}]

In [ ]:
aX[0].shape

(160, 160, 3)

In [ ]:
a = np.zeros(shape=(0, 160, 160, 3))
b = np.zeros(shape=(160, 160, 3)).reshape(1,160,160,3)

In [ ]:
c = np.vstack((a,b))

In [ ]:
c.shape

(1, 160, 160, 3)

In [ ]:
image = Image.open("faces/Chou_Tzuyu/chou_tzuyu_1.jpg")
face_array = np.asarray(image).reshape(1,160,160,3)

In [ ]:
face_array.shape

(1, 160, 160, 3)

In [ ]:
name = "Chou_Tzuyu/"
L = listdir("full/" + name)
L.sort()
len(L)

1836

In [ ]:
# enumerate files

for filename in L[1400:1600]:
  # path
  path = "full/" + name + filename
  # get face
  face = extract_face(path)
  if  not (isdir(name)) :
        mkdir(name)
  plt.imsave(name + filename,face)
  

In [ ]:
! zip -r -qq Chou_Tzuyu_1600.zip Chou_Tzuyu/

In [ ]:
! rm -r Chou_Tzuyu/

In [ ]:
len(listdir("Chou_Tzuyu/"))

400

In [ ]:
L[1400:1600][-1]

'chou_tzuyu_786.jpg'

In [ ]:
import cv2

In [ ]:
img = cv2.imread('faces/Myoui_Mina/myoui_mina_327.png')

In [ ]:
type(img)

numpy.ndarray

In [ ]:
cv2.imshow("abc", img)

DisabledFunctionError: ignored

DisabledFunctionError: ignored

DisabledFunctionError: ignored

DisabledFunctionError: ignored

In [ ]:
img2 = Image.open('faces/Myoui_Mina/myoui_mina_327.png')

In [ ]:
img1 = np.asarray(img2)

In [ ]:
img1.shape

(160, 160, 3)

In [ ]:
plt.imshow(img1)